In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, TensorDataset

from typing import List, Tuple, NoReturn, Any, Optional, Union
import pandas as pd
import numpy as np
import random

from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from tqdm.auto import tqdm, trange
import os

import datasets
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, HfArgumentParser
import re

In [2]:
dataset = load_from_disk("/opt/ml/data/train_dataset")
train_dataset = dataset["train"]
valid_dataset = dataset["validation"]
dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [3]:
df = valid_dataset.to_pandas()
df.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,전효숙,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...",처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"{'answer_start': [284], 'text': ['한보철강']}",9027,2146
1,스토우빌선,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,"{'answer_start': [146], 'text': ['1871년']}",51765,3106
2,촐라,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...",촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"{'answer_start': [517], 'text': ['나뭇잎']}",15694,1180
3,백조 기사,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,"{'answer_start': [1109], 'text': ['금대야']}",59536,141
4,탈관료제화,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,"{'answer_start': [386], 'text': ['수평적 관계']}",32991,184


In [4]:
test = df.question.tolist()
test

['처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?',
 '스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?',
 '촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?',
 '로타이르가 백조를 구하기 위해 사용한 것은?',
 '의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?',
 '1945년 쇼와천황의 항복 선언이 발표된 라디오 방송은?',
 '징금수는 서양 자수의 어떤 기법과 같은 기술을 사용하는가?',
 '다른 과 의사들은 감염내과 전문의들로부터 어떤 것에 대해 조언을 받는가?',
 '루이 14세의 왕비 마리아 테래사는 어느 나라 공주인가?',
 '헤자즈 왕국이 실존했던 것은 언제까지인가?',
 '버드 교장이 5월의 여왕의 대안으로 제시한 것은?',
 "인형사'를 만들어낸 것으로 추측되는 사업의 이름은?",
 '멘데스가 요원들을 구하기 위해 간 도시는 어디인가?',
 '교과부의 행동에 화가나 여러명이 사직한 기구의 이름은?',
 '반대동맹이 공산당과 갈라서겠다고 얘기한 날은 언제인가?',
 '피어슨이 다시 의회를 해산했던 년도는?',
 '몽케가 죽은 뒤 쿠릴타이에서 대칸의 지위를 얻은 사람의 이름은?',
 '이흥구의 사법시험 이야기를 기사로 작성한 곳은?',
 '남북조 시대에서 이이 씨가 전쟁이 발생했을 때, 생활했던 장소는?',
 '박지훈은 1라운드에서 몇 순위를 차지했는가?',
 '데메카론에는 무엇을 풍자하는 이야기가 들어있나요?',
 '병에 걸려 죽을 확률이 약 25~50%에 달하는 유형의 질병은?',
 '설리반이 불만을 표시한 대상은 누구인가?',
 '베소스는 어디서 추방당했는가?',
 '진전사의 명칭이 드러나는 데 영향을 준 물건은?',
 '자신의 이상적인 국가관이 스파르타와 닮아 있다고 생각하는 플라톤의 저서는?',
 '박제된 북극곰이 사망한 날짜는?',
 '문법 측면에서 더 보수적인 포르투갈어 표준은?',
 '로스 수장이 살해한 사람은 어느 당 회원인가?',
 '조경숙왕의 아들인 요자의 친어머니는 누구

In [6]:
df.context = test
df.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,전효숙,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"{'answer_start': [284], 'text': ['한보철강']}",9027,2146
1,스토우빌선,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,"{'answer_start': [146], 'text': ['1871년']}",51765,3106
2,촐라,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"{'answer_start': [517], 'text': ['나뭇잎']}",15694,1180
3,백조 기사,로타이르가 백조를 구하기 위해 사용한 것은?,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,"{'answer_start': [1109], 'text': ['금대야']}",59536,141
4,탈관료제화,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,"{'answer_start': [386], 'text': ['수평적 관계']}",32991,184


In [3]:
model_checkpoint = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [42]:
df = pd.read_csv("/opt/ml/data/bm25/bm25_top5_pp1_val.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/opt/ml/data/bm25/bm25_top5_pp1_val.csv'

In [35]:
answers = df.answers.to_list()
question = df.question.to_list()
context_id = df.context_id.to_list()
original_context = df.original_context.to_list()
top_k_passage = []
target = []

In [36]:
len(answers), len(question), len(original_context)

(240, 240, 240)

In [37]:
for idx, passages in enumerate(df.context.tolist()):
    p_list = passages.split('▦')
    
    if original_context[idx] in p_list:
        top_k_passage.append(p_list)
        target_index = p_list.index(original_context[idx])
        target.append(target_index)
    else :
        p_list = [original_context[idx]] + p_list[:-1]
        top_k_passage.append(p_list)
        target.append(0)

In [39]:
original_context[0]

'순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\\n\\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. \\n\\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.'

In [11]:
file_name = "dksahjklvjxzckl \
asdkljklzxjckljvcxzk \
ksdajklxzcvjklvxcz\
fjklasdjkldscv\
dfsjkljxckzl"

In [12]:
file_name

'dksahjklvjxzckl asdkljklzxjckljvcxzk ksdajklxzcvjklvxczfjklasdjkldscvdfsjkljxckzl'

In [5]:
train_custom_pickle = os.path.join(
    "/opt/ml/data/train_pickle", f"bm25_top10_pp1.pickle"
    )
valid_custom_pickle = os.path.join(
    "/opt/ml/data/valid_pickle", f"bm25_top10_pp1.pickle"
    )

In [6]:
import pickle
with open(train_custom_pickle, "rb") as f:
    train_dataset = pickle.load(f)
with open(valid_custom_pickle, "rb") as f:
    valid_dataset = pickle.load(f)

In [8]:
valid_dataset['context']

KeyError: 'context'

In [9]:
import random
i = random.randrange(240)
i

181

In [10]:
train_dataset['question'][i], valid_dataset['question'][i]

('엘리노어 여사가 사망한 해는?', '동궁전 화재에서 인종을 구한 사람은 누구인가?')

In [11]:
train_dataset['answers'][i], valid_dataset['answers'][i]

({'answer_start': [584], 'text': ['2007년']},
 {'answer_start': [285], 'text': ['숙의 정씨']})

In [13]:
train_dataset['target'][i], valid_dataset['target'][i]

(3, 0)

In [14]:
train_dataset['original_context'][i], valid_dataset['original_context'][i]

('1972년 패주 후에 맥거번은 상원에 자신의 의석으로 복귀하였다. 그는 자유주의 직위들을 위하여 지속적으로 웅변적이고 변덕스러운 옹호자였다. 10년의 세월들 동안 민주당의 당수들은 1972년 대선 운동과 선거를 두고 논쟁하였다. 그 일은 맥거번의 선거 운동으로부터 민주당원들 중에서 그와 거리를 두는 표준이 되었다. 맥거번은 자신이 재선을 위한 입찰을 패했던 1980년까지 상원을 지냈다. 그는 퇴직에 활동적으로 남아 자신이 중요한 것으로 믿은 문제들에 쓰고 연설하였다. 1994년 맥거번과 그의 부인 엘리노어 여사는 알콜중독으로부터 고통을 겪은 그들의 성인 딸인 테리가 그녀의 자동차에서 얼어 죽었을 때 비극을 견디었다. 자신의 큰 슬픔을 극복하는 데 맥거번은 〈테리:알콜중독과 함께 분투한 나의 딸의 인생과 사망〉 이라는 책을 저서하였다. 그는 그러고나서 알콜과 마약 중독에 옹호자가 되어 연설하였다. 빌 클린턴 대통령은 맥거번을 식량 농업 기구의 미국 대사로 임명하였다. 케네디 행정부에서 자신의 업무의 30년 후에 그는 식량과 기아 문제들에 다시 옹호하였다. 맥거번과 그의 부인은 사우스다코타주로 돌아왔다. 부인은 2007년에 사망하였다. 퇴직에서 활동으로 남아있던 맥거번은 자신의 88세 생일에 스카이 다이빙을 하러 갔다. 2012년 10월 21일 수폴스에서 90세의 나이로 사망하였다.',
 '한번은 동궁에 불이 붙었는데 빈궁이 옆에서 잠자다 깨어 발을 구르면서 얼른 뛰쳐나가자고 세자에게 애원하였다. 그러나 인종은 동궁에 불이난 것이 계모인 문정왕후의 뜻이라고 짐작하고는 빈궁보고만 먼저 나가라고 하였다 때마침 중종이 자신을 애타게 부르는 소리를 듣고서야 죽는 게 아버지에게 불효하는 것이라는 생각이 들어 불길을 헤치고 나왔다는 것이다 그러나 이는 문정왕후를 의도적으로 깎아내리는 야사일 뿐이고, 실제로 동궁전 화재 당시 인종은 깊게 잠들어 화재를 피하지 못하였는데 화재 현장을 목격한 인종의 후궁인 숙의 정씨가 직접 불길 속으로 뛰어 들어가 인종을 구했다고 한다. 추후

In [15]:
train_dataset['top_k_passage'][i], valid_dataset['top_k_passage'][i]

(['플로리다주 잭슨빌에서 6명의 자식들 중의 하나로 태어났다. 그녀가 2세 때 그녀의 부모가 사망하였다. 부모의 사망에 이어 새비지와 그녀의 형제, 자매들은 뉴저지주로 이주한 그녀의 고모와 함께 살러 갔다. 10세의 나이로 새비지는 뉴저지 주립 유색 여성 연맹을 대표하여 영부인에게 꽃을 전하는 데 선택되었을 때 엘리노어 루스벨트 여사를 만났다. 아무것도 말하지 말라고 가르쳤어도 새비지는 루스벨트 여사의 일생의 나머지를 위하여 둘이 펜팔이 되는 데 이끈 것에 그녀에게 감사를 표하였다. 12세의 나이로 그녀는 전미 흑인 지위 향상 협회에 가입하였다.',
  '프랭클린 D. 루스벨트가 1933년 대통령 임기를 시작할 때 엘리너 여사는 드라마적으로 영부인의 역할을 바꾸었다. 배경에 머물고 국내적 문제들을 다루는 데 만족하지 않은 그녀는 전국적으로 강의를 위한 만유를 하고 여성 리포터들 만을 위하여 350회의 기자 회견을 가졌으며, 일간 신문의 본란과 잡지들을 위한 많은 문서들을 썼다. 또한 젊은이들과 혜택받지 못한 사람들과 함께 소수 민족들을 위한 인권 투쟁을 벌이기도 하였다. 제2차 세계 대전이 일어나는 동안 미군들을 방문하러 세계를 순회하였다. 그녀는 1945년 4월 12일 프랭클린의 사망 때까지 영부인의 역할을 맡았다. == 유엔과 대통령의 임명들 == 남편의 사망에 이어 엘리너 여사는 기자 회견에서 자신이 공공적 서비스를 지속할 계획들이 없다고 말하였다. 하지만 해리 트루먼 대통령은 엘리너 여사를 유엔 총회의 사절단으로 임명하여 그녀는 그 직에 1945년부터 1953년까지 지냈다. 그녀는 유엔인권이사회의 의장이 되어 세계인권선언을 쓰는 도움을 주었으며 자신의 거대한 성과로 숙고한 노력이었다. 존 F. 케네디 대통령은 1961년 그녀를 유엔의 미국 사절단으로 재임명하였고, 후에 미국 평화 봉사단의 국가 안보 위원회와 대통령 여성 지위 자문회의 의장으로 임명하였다.',
  '엘리사베타는 만토바 후작 페데리코 1세 곤차가와 마르가레테 폰 바이에른 사이에서 둘째 딸

In [1]:
import pickle
with open("/opt/ml/data/valid_pickle/bm25_top10_pp1.pickle", "rb") as f:
    v = pickle.load(f)

In [6]:
v['original_context'][0], v['target'][0], len(v['top_k_passage'][0])

('순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다. 경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. 2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.',
 0,
 10)

In [ ]:
DPR_passage['original_context'][0]
DPR_passage['context'][0]

In [15]:
from glob import glob

file_list = glob('/opt/ml/mrc-level2-nlp-02/retriever/encoders/*')

In [16]:
file_list

['/opt/ml/mrc-level2-nlp-02/retriever/encoders/bert-base_ep2_bs2_topk20_acs8_pp1_lr5e-05',
 '/opt/ml/mrc-level2-nlp-02/retriever/encoders/roberta-large_ep4_bs1_topk10_acs1_pp2_lr1e-05',
 '/opt/ml/mrc-level2-nlp-02/retriever/encoders/bert-base_ep2_bs1_topk10_acs8_pp1_lr5e-05']

In [18]:
for idx, file_name in   (file_list):
    print(f"{idx} : {file_name.split('/')[-1]}")

0 : bert-base_ep2_bs2_topk20_acs8_pp1_lr5e-05
1 : roberta-large_ep4_bs1_topk10_acs1_pp2_lr1e-05
2 : bert-base_ep2_bs1_topk10_acs8_pp1_lr5e-05
